In [1]:
from reference_metric import *
reference_metric()
print(scalefactor_orthog[2])
xxhats = UnitVectors3D()
print(xxhats[0])
print(xxhats[1])
print(xxhats[2])

xx0*sin(xx1)
[sqrt(bScale**2 + xx0**2)*sin(xx1)*cos(xx2)/sqrt(bScale**2*sin(xx1)**2 + xx0**2), sqrt(bScale**2 + xx0**2)*sin(xx1)*sin(xx2)/sqrt(bScale**2*sin(xx1)**2 + xx0**2), xx0*cos(xx1)/sqrt(bScale**2*sin(xx1)**2 + xx0**2)]
[xx0*cos(xx1)*cos(xx2)/sqrt(bScale**2*sin(xx1)**2 + xx0**2), xx0*sin(xx2)*cos(xx1)/sqrt(bScale**2*sin(xx1)**2 + xx0**2), -sqrt(bScale**2 + xx0**2)*sin(xx1)/sqrt(bScale**2*sin(xx1)**2 + xx0**2)]
[-sin(xx1)*sin(xx2)/Abs(sin(xx1)), sin(xx1)*cos(xx2)/Abs(sin(xx1)), 0]


In [2]:
xx0,xx1 = sp.symbols('xx0 xx1',positive=True)
xx0/sp.sqrt(xx0**2)

1